In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,BatchNormalization
from tensorflow.keras.layers import Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


In [2]:
from tensorflow.keras.models import Sequential


In [3]:
# Set constants
IMAGE_SIZE = 48
BATCH_SIZE = 128
NUM_CLASSES = 7
EPOCHS = 100

In [4]:
# Define data generators for training and validation sets
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator= train_datagen.flow_from_directory(directory = '/kaggle/input/fer2013/train',
                                                    target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    color_mode='grayscale')

Found 28709 images belonging to 7 classes.


In [6]:
image_shape = train_generator.image_shape
image_shape

(48, 48, 1)

In [7]:
val_generator = test_datagen.flow_from_directory(directory = '/kaggle/input/fer2013/test',
                                                    target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical',
                                                    color_mode='grayscale')

Found 7178 images belonging to 7 classes.


In [8]:
image_shape = val_generator.image_shape
image_shape

(48, 48, 1)

In [9]:
def inception_module(layer_in, f1, f2, f3):
# 1x1 conv
 conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
 # 3x3 conv
 conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
 # 5x5 conv
 conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
 # 3x3 max pooling
 pool = MaxPooling2D((2,2), strides=(1,1), padding='same')(layer_in)
 # concatenate filters, assumes filters/channels last
 out = Concatenate(axis=-1)([conv1, conv3, conv5, pool])
 return out

In [10]:
input_layer = Input(shape=image_shape)

x = Conv2D(32,(3,3),padding='same', activation='relu')(input_layer)
x = MaxPooling2D((2,2),strides = (1,1),padding='same')(x)
x = Dropout(0.2)(x)

x = BatchNormalization()(x)

x = Conv2D(64,(5,5),padding='same', activation='relu')(input_layer)
x = MaxPooling2D((2,2),strides = (1,1),padding='same')(x)
x = Dropout(0.2)(x)

x = BatchNormalization()(x)

x = inception_module(x, 64, 128, 32)
x = Dropout(0.2)(x)
x = inception_module(x, 64, 128, 32)
x = Dropout(0.2)(x)
x = inception_module(x, 64, 128, 32)

x = Flatten()(x)

dense = Dense(128,activation='LeakyReLU')(x)
x = Dropout(0.2)(x)
dense = Dense(128,activation='relu')(x)
out_layer = Dense(7,activation='softmax')(x)
 

model = Model(input_layer,out_layer)


In [11]:
adam =tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam'
    
)

In [12]:
model.compile(optimizer=adam,
               loss="categorical_crossentropy", 
             metrics=['accuracy'])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 48, 48, 1)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 48, 48, 64)   1664        ['input_1[0][0]']                
                                                                                                  
 max_pooling2d_1 (MaxPooling2D)  (None, 48, 48, 64)  0           ['conv2d_1[0][0]']               
                                                                                                  
 dropout_1 (Dropout)            (None, 48, 48, 64)   0           ['max_pooling2d_1[0][0]']        
                                                                                              

In [13]:
earlystopping =tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                patience=5, verbose=1, mode="auto", )
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8,
                              verbose=1, mode='max', min_lr=5e-5)
checkpoint = tf.keras.callbacks.ModelCheckpoint('version1_finetune.h15', monitor= 'val_accuracy',
                             mode= 'max', save_best_only = True, verbose= 1)

In [14]:
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=val_generator,
                    validation_steps=val_generator.samples // BATCH_SIZE)

Epoch 1/100


2023-03-01 10:50:06.825879: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout_1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


224/224 [==============================] - 207s 864ms/step - loss: 7.8797 - accuracy: 0.2038 - val_loss: 1.8902 - val_accuracy: 0.2020
Epoch 2/100
224/224 [==============================] - 86s 384ms/step - loss: 1.8813 - accuracy: 0.2318 - val_loss: 1.8435 - val_accuracy: 0.2492
Epoch 3/100
224/224 [==============================] - 89s 398ms/step - loss: 1.8165 - accuracy: 0.2448 - val_loss: 1.7860 - val_accuracy: 0.2778
Epoch 4/100
224/224 [==============================] - 90s 399ms/step - loss: 1.8006 - accuracy: 0.2521 - val_loss: 1.7803 - val_accuracy: 0.2778
Epoch 5/100
224/224 [==============================] - 89s 398ms/step - loss: 1.7905 - accuracy: 0.2592 - val_loss: 1.7895 - val_accuracy: 0.2638
Epoch 6/100
224/224 [==============================] - 89s 398ms/step - loss: 1.7882 - accuracy: 0.2595 - val_loss: 1.7701 - val_accuracy: 0.2811
Epoch 7/100
224/224 [==============================] - 89s 399ms/step - loss: 1.7823 - accuracy: 0.2616 - val_loss: 1.7695 - val_accura

In [15]:
 model.save("Version1.h5")


In [16]:
import pandas as pd
new = pd.DataFrame.from_dict(history.history)
new.to_csv("inception_dset1_v2.csv")

In [17]:
loss, acc = model.evaluate(val_generator, verbose=1)


57/57 [==============================] - 8s 130ms/step - loss: 1.3910 - accuracy: 0.4808
